In [14]:
%pip install geopandas pandas matplotlib folium numpy

import os
import geopandas as gpd
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import seaborn as sns
from subprocess import check_output

Note: you may need to restart the kernel to use updated packages.


In [38]:
#Ensure that the data is in the correct directory
data_path = "data/crime/"
if os.path.exists(data_path):
    print("Found the directory! Here's what's inside:")
    print(os.listdir(data_path))
else:
    print("Directory not found. Check your map and compass!")

# Assuming the directory exists, let's proceed to load the files
file_names = [
    "CrimeData-00-2018.csv",
    "CrimeData-2018.csv",
    "CrimeData-2019.csv",
    "CrimeData-2020.csv",
    "CrimeData-2021.csv",
    "CrimeData-2022.csv",
    "CrimeData-2023.csv"
]

# Correctly construct the full file paths
all_files = [os.path.join(data_path, file_name) for file_name in file_names]

# Load and concatenate the CSV files into a single DataFrame
try:
    crimes = pd.concat((pd.read_csv(f) for f in all_files))
    # Display the first three rows to verify the data
    print(crimes.head(3))
except FileNotFoundError as e:
    print(f"Error loading files: {e}")

Found the directory! Here's what's inside:
['CrimeData-00-2018.csv', 'CrimeData-2015.csv', 'CrimeData-2016.csv', 'CrimeData-2018.csv', 'CrimeData-2019.csv', 'CrimeData-2020.csv', 'CrimeData-2021.csv', 'CrimeData-2022.csv', 'CrimeData-2023.csv', 'merged_crimes.csv', 'Neighborhood_Boundaries.cpg', 'Neighborhood_Boundaries.dbf', 'Neighborhood_Boundaries.geojson', 'Neighborhood_Boundaries.prj', 'Neighborhood_Boundaries.shp', 'Neighborhood_Boundaries.shx']
                       Address Case Number Crime Against  Neighborhood  Number of Records Occur Date Occur Month Year  Occur Time  Offense Category  Offense Count              Offense Type  OpenDataLat  OpenDataLon  OpenDataX  OpenDataY Report Date ReportMonthYear CaseNumber CrimeAgainst OccurDate  OccurTime OffenseCategory OffenseType ReportDate  OffenseCount
0    3600 BLOCK OF SE KNAPP ST   17-902332      Property  Eastmoreland                1.0  2/20/2017         2/1/2017         0.0  Larceny Offenses            1.0  Theft From Motor 

In [40]:
def int_to_strtime(d):
    """convert int hhmm to str HH:MM """
    # Ensure that the input is a string with leading zeros if necessary
    d = '{:0>4}'.format(d)
    (h, s) = (d[:2], d[2:])
    timestr = h + ':' + s
    return timestr

# Ensure 'OccurTime' is treated as a string and then convert to HH:MM format
crimes['OccurTime'] = crimes['OccurTime'].astype(str).apply(lambda x: '{:0>4}'.format(x))
crimes['Occur_time'] = crimes['OccurTime'].apply(int_to_strtime)

# Note: Adjust 'OccurDate' to match the exact case and spelling of your date column
crimes['Occur_timestamp'] = pd.to_datetime(crimes['OccurDate'] + ' ' + crimes['Occur_time'], errors='coerce')

# Extracting components
crimes['Hour'] = crimes['Occur_timestamp'].dt.hour
crimes['Date'] = crimes['Occur_timestamp'].dt.date
crimes['Day_of_Week'] = crimes['Occur_timestamp'].dt.dayofweek

C:\Users\asher\AppData\Local\Temp\ipykernel_22912\532167503.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  crimes['Occur_timestamp'] = pd.to_datetime(crimes['OccurDate'] + ' ' + crimes['Occur_time'], errors='coerce')


In [1]:
# Specify your desired output file name
output_file_path = data_path + '/final_crimes.csv'

# Export the DataFrame to a CSV file
crimes.to_csv(output_file_path, index=False)

NameError: name 'data_path' is not defined

take a look at the neighborhood names in both your crime data (CSV) and your neighborhood shapefile (GeoDataFrame) to understand the formatting and any potential inconsistencies.

In [42]:
# Load crime data
crimes_df = pd.read_csv('data/crime/final_crimes.csv')

# Load neighborhood shapefile
neighborhoods_gdf = gpd.read_file('data/crime/Neighborhood_Boundaries.shp')

# Assuming 'crimes_df' is your DataFrame and 'neighborhoods_gdf' is your GeoDataFrame
print('CSV Names: ', crimes_df['Neighborhood'].unique())
print('SHP Names: ',  neighborhoods_gdf['MAPLABEL'].unique())


C:\Users\asher\AppData\Local\Temp\ipykernel_22912\3294836310.py:2: DtypeWarning: Columns (1,2,5,6,8,10,15,16,17,18,19,20,21,22,23,26,28) have mixed types. Specify dtype option on import or set low_memory=False.
  crimes_df = pd.read_csv('data/crime/final_crimes.csv')


CSV Names:  ['Eastmoreland' 'Eliot' 'Far Southwest' 'Forest Park' 'Foster-Powell'
 'Glenfair' 'Goose Hollow' 'Grant Park' 'Hayden Island' 'Hazelwood'
 'Portsmouth' 'Overlook' 'Parkrose' 'Parkrose Heights' 'Pearl' 'Piedmont'
 'Pleasant Valley' 'Powellhurst-Gilbert' 'Reed' 'Richmond'
 'Rose City Park' 'Roseway' 'Russell' 'Sellwood-Moreland' 'South Portland'
 'South Tabor' 'St Johns' "Sullivan's Gulch" 'Sumner' 'Sunderland'
 'Sunnyside' 'University Park' 'West Portland Park' 'Wilkes' 'Woodlawn'
 'Woodstock' nan 'Arbor Lodge' 'Argay' 'Arnold Creek' 'Ashcreek'
 'Beaumont-Wilshire' 'Boise' 'Brentwood-Darlington' 'Bridgeton'
 'Bridlemile' 'Brooklyn' 'Buckman East' 'Buckman West' 'Cathedral Park'
 'Centennial' 'Concordia' 'Creston-Kenilworth' 'Crestwood' 'Cully'
 'Hillsdale' 'Hillside' 'Hosford-Abernethy' 'Irvington' 'Kenton' 'Kerns'
 'King' 'Lents' 'Linnton' 'Lloyd' 'Marshall Park' 'Mill Park' 'Montavilla'
 'Mt Scott-Arleta' 'Mt Tabor' 'North Tabor' 'Northwest'
 'Old Town/Chinatown' 'Sabin' '

To ensure a successful match, convert all neighborhood names to a consistent case (e.g., all lowercase) and strip any leading or trailing spaces.

In [43]:
crimes_df['Neighborhood'] = crimes_df['Neighborhood'].str.lower().str.strip()
neighborhoods_gdf['MAPLABEL'] = neighborhoods_gdf['MAPLABEL'].str.lower().str.strip()

If you notice spelling inconsistencies or variations in how neighborhood names are written, you may need to manually map these variations to a standard form. Create a dictionary to map variant spellings to a standardized spelling and apply this mapping.

In [49]:
# Assuming 'neighborhood_column_in_csv' and 'neighborhood_column_in_shapefile' are the correct column names
unique_crime_neighborhoods = set(crimes_df['Neighborhood'].unique())
unique_shapefile_neighborhoods = set(neighborhoods_gdf['MAPLABEL'].unique())

In [50]:
# Names in crime data not found in shapefile
missing_in_shapefile = unique_crime_neighborhoods - unique_shapefile_neighborhoods
# Names in shapefile not found in crime data
missing_in_crime_data = unique_shapefile_neighborhoods - unique_crime_neighborhoods

missing_in_shapefile_sorted = sorted([str(item) for item in missing_in_shapefile])
missing_in_crime_data_sorted = sorted([str(item) for item in missing_in_crime_data])

print("Names in crime data not found in shapefile:", missing_in_shapefile_sorted)
print("Names in shapefile not found in crime data:", missing_in_crime_data_sorted)

Names in crime data not found in shapefile: ['ardenwald', 'argay', 'buckman east', 'buckman west', 'downtown', 'healy heights', 'mt scott-arleta', 'mt tabor', 'nan', 'northwest', 'northwest industrial', 'old town/chinatown', 'pearl', 'st johns']
Names in shapefile not found in crime data: ['alameda/beaumont-wilshire', 'alameda/irvington', 'ardenwald-johnson creek', 'ardenwald-johnson creek/woodstock', 'argay terrace', 'argay terrace/wilkes', 'arlington heights/sylvan-highlands', 'ashcreek/crestwood', 'boise/eliot', 'bridlemile/southwest hills', 'buckman', 'centennial/pleasant valley', 'eastmoreland/ardenwald-johnson creek', 'eastmoreland/reed', 'forest park/linnton', 'forest park/northwest district', 'goose hollow/southwest hills', 'grant park/hollywood', 'hazelwood/mill park', 'healy heights/southwest hills', 'hillside/northwest district', 'lents/powellhurst-gilbert', "lloyd/sullivan's gulch", 'mc unclaimed #11', 'mc unclaimed #13', 'mc unclaimed #14', 'mc unclaimed #5', 'mt. scott-ar

Create a Mapping Dictionary

In [34]:
name_mapping = {
    'ardenwald': 'ardenwald-johnson creek',
    'argay': 'argay terrace',
    'buckman east': 'buckman',  # Assuming you want to merge east/west
    'buckman west': 'buckman',  # Assuming you want to merge east/west
    'downtown': 'portland downtown',
    'healy heights': 'healy heights/southwest hills',
    'mt scott-arleta': 'mt. scott-arleta',
    'mt tabor': 'mt. tabor',
    'northwest': 'northwest district',
    'northwest industrial': 'northwest district',  # Assuming you want to merge; adjust based on your analysis
    'old town/chinatown': 'old town',  # Decide based on which name is more encompassing or split entries
    'pearl': 'pearl district',
    'st johns': 'st. johns',
    'nan': None  # Decide how to handle 'nan'; perhaps exclude these records or map them to a placeholder
}

Apply the Mapping

In [52]:
crimes_df['Neighborhood'] = crimes_df['Neighborhood'].map(name_mapping).fillna(crimes_df['Neighborhood'])

Re-check and Adjust
After applying the initial mapping, re-check for any names that still don't match and adjust your mapping dictionary accordingly. This step might require iterating a few times to catch all discrepancies.

In [53]:
# Assuming 'neighborhood_column_in_csv' and 'neighborhood_column_in_shapefile' are the correct column names
unique_crime_neighborhoods = set(crimes_df['Neighborhood'].unique())
unique_shapefile_neighborhoods = set(neighborhoods_gdf['MAPLABEL'].unique())
# Names in crime data not found in shapefile
missing_in_shapefile = unique_crime_neighborhoods - unique_shapefile_neighborhoods
# Names in shapefile not found in crime data
missing_in_crime_data = unique_shapefile_neighborhoods - unique_crime_neighborhoods

missing_in_shapefile_sorted = sorted([str(item) for item in missing_in_shapefile])
missing_in_crime_data_sorted = sorted([str(item) for item in missing_in_crime_data])

print("Names in crime data not found in shapefile:", missing_in_shapefile_sorted)
print("Names in shapefile not found in crime data:", missing_in_crime_data_sorted)

Names in crime data not found in shapefile: ['nan']
Names in shapefile not found in crime data: ['alameda/beaumont-wilshire', 'alameda/irvington', 'ardenwald-johnson creek/woodstock', 'argay terrace/wilkes', 'arlington heights/sylvan-highlands', 'ashcreek/crestwood', 'boise/eliot', 'bridlemile/southwest hills', 'centennial/pleasant valley', 'eastmoreland/ardenwald-johnson creek', 'eastmoreland/reed', 'forest park/linnton', 'forest park/northwest district', 'goose hollow/southwest hills', 'grant park/hollywood', 'hazelwood/mill park', 'hillside/northwest district', 'lents/powellhurst-gilbert', "lloyd/sullivan's gulch", 'mc unclaimed #11', 'mc unclaimed #13', 'mc unclaimed #14', 'mc unclaimed #5', 'pleasant valley/powellhurst-gilbert', 'roseway/madison south', 'sabin/irvington', "sullivan's gulch/grant park", 'sylvan-highlands/southwest hills']


Perform the Join and Review the Data

In [54]:
# Perform an attribute join
joined_gdf = neighborhoods_gdf.merge(crimes_df, how='left', left_on='MAPLABEL', right_on='Neighborhood')

# Preview the first few rows
print(joined_gdf.head())

# Check for missing values in key columns, adjust column names as needed
print(joined_gdf[['MAPLABEL', 'Neighborhood']].isnull().sum())

   OBJECTID     NAME COMMPLAN SHARED COALIT HORZ_VERT    Shape_Leng MAPLABEL  ID                                           geometry                          Address Case Number Crime Against Neighborhood  Number of Records Occur Date Occur Month Year  Occur Time  Offense Category  Offense Count              Offense Type  OpenDataLat  OpenDataLon  OpenDataX  OpenDataY Report Date ReportMonthYear CaseNumber CrimeAgainst OccurDate OccurTime OffenseCategory OffenseType ReportDate  OffenseCount Occur_time Occur_timestamp  Hour Date  Day_of_Week
0         1  LINNTON     None      N   NWNW      HORZ  53244.045538  linnton   1  POLYGON ((-13669901.863 5719574.412, -13669574...  11300 BLOCK OF NW GERMANTOWN RD   18-148935      Property      linnton                1.0   5/5/2018         5/1/2018      1600.0    Fraud Offenses            1.0            Identity Theft    45.589627  -122.790542  7615808.0   709376.0    5/5/2018        May 2018        NaN          NaN       NaN      0nan             

Save as a geojson file

In [55]:
# Step 4: Save as GeoJSON
joined_gdf.to_file("data/pdx_crime.geojson", driver='GeoJSON')